In [1]:
import h5py
import numpy as np
import pandas as pd

from scipy.ndimage import zoom
from cellpose import models
from cellstitch.evaluation import *

In [2]:
test_filenames = ["N_294_final_crop_ds2.npy", 
                 "N_435_final_crop_ds2.npy",
                 "N_441_final_crop_ds2.npy",
                 "N_511_final_crop_ds2.npy",
                 "N_522_final_crop_ds2.npy",
                 "N_590_final_crop_ds2.npy",
                 "N_593_final_crop_ds2.npy"]
anisotropy = 4

## Generate images

First, generate and store the interpolated images.

In [4]:
for test_filename in test_filenames:
    print("Starting %s ..." % test_filename)
    img = np.load('../DATA/ovules/raw/%s' % test_filename)
    interp_img = zoom(img, (anisotropy, 1, 1))
    
    np.save(
        "../DATA/ovules_interp/raw/%s" % test_filename, 
        interp_img
    )

Starting N_294_final_crop_ds2.npy ...
Starting N_435_final_crop_ds2.npy ...
Starting N_441_final_crop_ds2.npy ...
Starting N_511_final_crop_ds2.npy ...
Starting N_522_final_crop_ds2.npy ...
Starting N_590_final_crop_ds2.npy ...
Starting N_593_final_crop_ds2.npy ...


## Cellpose3d

Benchmark cellpose3d on interpolated images. 

In [4]:
model_dir = '../DATA/ovules/cellpose_train/models/cellpose_residual_on_style_on_concatenation_off_cellpose_train_2023_05_08_09_31_06.231473'
flow_threshold = 1
model = models.CellposeModel(gpu=True, pretrained_model=model_dir)

In [ ]:
for test_filename in test_filenames[1:]: 
    print("Starting %s" % test_filename)
    img = np.load("../DATA/ovules_interp/raw/%s" % test_filename)
    masks, _, _ = model.eval(img, do_3D=True, flow_threshold=flow_threshold, channels = [0,0])
    np.save("./results/ovules_interp/%s" % test_filename, masks)

Starting N_435_final_crop_ds2.npy


In [5]:
for test_filename in test_filenames[0:]: 
    img = np.load("../DATA/ovules_interp/raw/%s" % test_filename)
    print(test_filename, img.shape)

N_294_final_crop_ds2.npy (1280, 960, 1000)
N_435_final_crop_ds2.npy (1552, 1101, 1110)
N_441_final_crop_ds2.npy (1776, 1095, 1028)
N_511_final_crop_ds2.npy (1040, 810, 715)
N_522_final_crop_ds2.npy (1480, 810, 935)
N_590_final_crop_ds2.npy (680, 555, 770)
N_593_final_crop_ds2.npy (560, 480, 1203)


Evaluate: 

In [10]:
data = []

for test_filename in ["N_294_final_crop_ds2.npy", 
                      "N_511_final_crop_ds2.npy",
                      "N_522_final_crop_ds2.npy",
                      "N_590_final_crop_ds2.npy",
                      "N_593_final_crop_ds2.npy"]:
    print("Starting %s" % test_filename) 
    labels = np.load('../DATA/ovules/labels/%s' % test_filename)
    masks = np.load("./results/ovules_interp/%s" % test_filename)
    masks = masks[::anisotropy,] # downsample to original size
    
    ap25, _, _, _ = average_precision(labels, masks, 0.25)
    ap50, tp, fp, fn = average_precision(labels, masks, 0.5)
    ap75, _, _, _ = average_precision(labels, masks, 0.75) 

    if (tp + fp) != 0: 
        precision = tp / (tp + fp)
    else: 
        precision = 0

    if tp + fp != 0: 
        recall = tp / (tp + fn)
    else: 
        precision = 0

    row = [ 
        test_filename, 
        ap25,
        ap50,
        ap75,
        tp, 
        fp, 
        fn, 
        precision,
        recall
    ]

    data.append(row)

Starting N_294_final_crop_ds2.npy
Starting N_511_final_crop_ds2.npy
Starting N_522_final_crop_ds2.npy
Starting N_590_final_crop_ds2.npy
Starting N_593_final_crop_ds2.npy


NameError: name 'pd' is not defined

In [12]:
df = pd.DataFrame(data, columns=[
        "filename",
        "ap25", 
        "ap50",
        "ap75", 
        "tp", 
        "fp", 
        "fn",
        "precision",
        "recall"
    ])

df.to_csv("./results/ovules_interp/cellpose3d.csv", index=False)

## PlantSeg

In [19]:
for test_filename in test_filenames: 
    img = np.load("../DATA/ovules_interp/raw/%s" % test_filename)
        
    with h5py.File("../DATA/ovules_interp/plantseg_test/%s.h5" % test_filename[0:-4], 'w') as hf:
        hf.create_dataset("raw",  data=img)

- activate the environment: `conda activate plant-seg` 
- set the `path` in `config.yaml` to `../DATA/ovules_interp/plantseg_test/` 
- perform segmentation with the `confocal_3D_unet_ovules_ds1x` by running `plantseg --config config.yaml`

In [3]:
plantseg_results_folder = "../DATA/ovules_interp/plantseg_test/PreProcessing/confocal_3D_unet_ovules_ds1x/MultiCut"

In [4]:
for test_filename in test_filenames: 
    print("Starting %s" % test_filename) 
    
    with h5py.File("%s/%s_predictions_multicut.h5" % (plantseg_results_folder, test_filename[:-4]), "r") as f:
        plantseg = np.array(list(f['segmentation'])) 
        
    plantseg[np.where(plantseg == 1)] = 0 # plantseg use 1 as labels
    np.save("./results/ovules_interp/plantseg/%s" % test_filename, plantseg)

Starting N_294_final_crop_ds2.npy
Starting N_435_final_crop_ds2.npy
Starting N_441_final_crop_ds2.npy
Starting N_511_final_crop_ds2.npy
Starting N_522_final_crop_ds2.npy
Starting N_590_final_crop_ds2.npy
Starting N_593_final_crop_ds2.npy


In [5]:
data = []

for test_filename in test_filenames:
    print("Starting %s" % test_filename) 
    labels = np.load('../DATA/ovules/labels/%s' % test_filename)
    masks = np.load("./results/ovules_interp/plantseg/%s" % test_filename)
    masks = masks[::anisotropy,] # downsample to original size
    
    ap25, _, _, _ = average_precision(labels, masks, 0.25)
    ap50, tp, fp, fn = average_precision(labels, masks, 0.5)
    ap75, _, _, _ = average_precision(labels, masks, 0.75) 

    if (tp + fp) != 0: 
        precision = tp / (tp + fp)
    else: 
        precision = 0

    if tp + fp != 0: 
        recall = tp / (tp + fn)
    else: 
        precision = 0

    row = [ 
        test_filename, 
        ap25,
        ap50,
        ap75,
        tp, 
        fp, 
        fn, 
        precision,
        recall
    ]

    data.append(row)

Starting N_294_final_crop_ds2.npy


/home/yining/anaconda3/envs/segmentation/lib/python3.8/site-packages/cellpose/metrics.py:205: RuntimeWarning: invalid value encountered in true_divide
  iou = overlap / (n_pixels_pred + n_pixels_true - overlap)


Starting N_435_final_crop_ds2.npy
Starting N_441_final_crop_ds2.npy
Starting N_511_final_crop_ds2.npy
Starting N_522_final_crop_ds2.npy
Starting N_590_final_crop_ds2.npy
Starting N_593_final_crop_ds2.npy


In [6]:
df = pd.DataFrame(data, columns=[
        "filename",
        "ap25", 
        "ap50",
        "ap75", 
        "tp", 
        "fp", 
        "fn",
        "precision",
        "recall"
    ])

df.to_csv("./results/ovules_interp/plantseg.csv", index=False)